## Issue of EM algorithm

The lack of analytical maximizer in M-step makes the EM algorithm pretty nasty. For instance, when it does not converge, it is hard to say whether it is because M-step fails or a bug in EM. A better solution is to use existing solver for M-step but I failed to find a one. 

**I am also waiting for an analytical result!**

### Using `lme4::lmer`

The straightforward idea is to use an out-of-box random effect solver. Well, the M-step problem can be formalized as a weighted random effect model (for $K = 2$).

$$\begin{align*}
    \hat\beta_{gwas} &= \beta_{gene, 1} \hat\beta_{eqtl, 1} + \beta_{gene, 2} \hat\beta_{eqtl, 2} + e \\
    \hat\beta_{eqtl, 1}, \hat\beta_{eqtl, 2} &= \begin{cases}
        \hat\beta_{eqtl}, 0 \\
        0, \hat\beta_{eqtl}
    \end{cases} \\
    \beta_{gene, 1} &\sim \mathcal{N}(0, \sigma_1^2) \\
    \beta_{gene, 2} &\sim \mathcal{N}(0, \sigma_2^2) \\
    e &\sim \mathcal{N}(0, \sigma^2)
\end{align*}$$

By doing this construction, it is almost solving the M-step problem and the only missing part is weight. Let $y = [\hat\beta_{gwas}, \hat\beta_{gwas}]$, $x_1 = [\hat\beta_{eqtl}, \vec{0}], x_2 = [\vec{0}, \hat\beta_{eqtl}], w = [w_{11}, \cdots, w_{n1}, w_{12}, \cdots, w_{n2}]$. The equivalent random effect model is

$$\begin{align*}
    y &= \beta_{gwas, 1} x_1 + \beta_{gwas, 2} x_2 + e 
\end{align*}$$
, with weighted likelihood 
$$\begin{align*}
    lld &= \sum_i \log w_i \Pr(y_i | x_1, x_2)
\end{align*}$$

Unfortunately, I find no way to use weighted likelihood in `lme4::lmer`. The `weights` option in it is for modeling heteroscedasticity. Nonetheless, `lme4::lmer` becomes a validation of my own M-step solver by applying to the case with equal weights. See [`compare_to_lmer.html`](http://htmlpreview.github.io/?https://raw.githubusercontent.com/liangyy/idea-playground/master/mix_random_effect/compare_to_lmer.html). 

**I am waiting for a better alternative RE solver!**


### EM solver performance

See [`run.html`](http://htmlpreview.github.io/?https://raw.githubusercontent.com/liangyy/idea-playground/master/mix_random_effect/run.html). Note that `idx` determines the initial value of EM algorithm. I generated 3 data sets with sample sizes 1e3, 1e4, 1e5. EM was run 10 times with 10 initial points. The solution is not satisfiable since it is not very stable across runs and the one with highest likelihood is not quite close to the truth. $\hat\sigma^2$ is fine, but $\hat\sigma_2^2$ (the bigger one) is quite problematic. And the worst thing is that $\hat\pi$ is not really trustable, which is kinda the only variable we really care about. So that it makes the whole framework seemingly useless.

